In [1]:
import os
import sys
from datetime import datetime
import numpy as np
from numpy.random import seed
from PIL import Image
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import backend as K
from tensorflow import set_random_seed
from tensorflow.keras.models import Model

from core.utils_nn import parse_file, load_gray_image, load_color_image, get_model_path, to_abs
from core.generator import ConfigurationGenerator, Generator
from core.pgd_attack import PGD_attack as pgd
import pickle

from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
input_shape = [256, 256, 1]
n_classes = 2
batch_size = 25
images_are_colored = False
train_file = 'train.txt'
test_file = 'test.txt'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
absp = '../__xray_age_new_20-35_50-70_'

In [4]:
model_path = get_model_path(absp, model_name='inceptionv3')

In [5]:
model = load_model(model_path)

W1103 23:32:47.390562 140288152303424 deprecation.py:506] From /home/dmitryvoynov/nn/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1103 23:32:47.391389 140288152303424 deprecation.py:506] From /home/dmitryvoynov/nn/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1103 23:32:47.395060 140288152303424 deprecation.py:506] From /home/dmitryvoynov/nn/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.o

In [7]:
feature_model = Model(inputs=model.input, outputs=model.layers[-2].output)

In [8]:
feature_model.output

<tf.Tensor 'avg_pool/Mean:0' shape=(?, 2048) dtype=float32>

In [5]:
layer_names = set([type(layer).__name__ for layer in model.layers])
layer_names

{'Activation',
 'AveragePooling2D',
 'BatchNormalization',
 'Concatenate',
 'Conv2D',
 'Dense',
 'GlobalAveragePooling2D',
 'InputLayer',
 'MaxPooling2D'}

In [6]:
not_used_layers = ['Concatenate', 'InputLayer', 'Dense']

In [7]:
layers_num = dict(zip(layer_names, [0] * len(layer_names)))

In [8]:
for layer_name in layers_num:
    layers_num[layer_name] = len([layer for layer in model.layers if type(layer).__name__ == layer_name])
layers_num

{'GlobalAveragePooling2D': 1,
 'Dense': 1,
 'Concatenate': 15,
 'InputLayer': 1,
 'MaxPooling2D': 4,
 'AveragePooling2D': 9,
 'Conv2D': 94,
 'Activation': 94,
 'BatchNormalization': 94}

In [9]:
used_layer_names = layer_names.difference(set(not_used_layers))
used_layer_names

{'Activation',
 'AveragePooling2D',
 'BatchNormalization',
 'Conv2D',
 'GlobalAveragePooling2D',
 'MaxPooling2D'}

In [10]:
each_layer_feature_model = Model(
    inputs=model.input,
    outputs=[l.output for l in model.layers if type(l).__name__ in used_layer_names]
)

In [11]:
def parse_shape(tf_shape):
    return np.array([int(d) for d in list(tf_shape)[1:]])

def get_shape(tf_layer):
    return parse_shape(tf_layer.shape)

In [12]:
outputs_shape = list(map(get_shape, each_layer_feature_model.outputs))
outputs_shape

[array([127, 127,  32]),
 array([127, 127,  32]),
 array([127, 127,  32]),
 array([125, 125,  32]),
 array([125, 125,  32]),
 array([125, 125,  32]),
 array([125, 125,  64]),
 array([125, 125,  64]),
 array([125, 125,  64]),
 array([62, 62, 64]),
 array([62, 62, 80]),
 array([62, 62, 80]),
 array([62, 62, 80]),
 array([ 60,  60, 192]),
 array([ 60,  60, 192]),
 array([ 60,  60, 192]),
 array([ 29,  29, 192]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 48]),
 array([29, 29, 96]),
 array([29, 29, 48]),
 array([29, 29, 96]),
 array([29, 29, 48]),
 array([29, 29, 96]),
 array([ 29,  29, 192]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 96]),
 array([29, 29, 32]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 96]),
 array([29, 29, 32]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 96]),
 array([29, 29, 32]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 64]),
 array([29, 29, 48]),
 array([29, 

In [13]:
np.sum([np.sum(shape) for shape in outputs_shape])

74256

In [14]:
paths, labels = parse_file(os.path.join(absp, 'train.txt'))

In [30]:
imgs = list(map(Image.open, paths[0: 10]))

In [32]:
imgs = np.array([np.array(img)[:, :, :3] for img in imgs])
features = each_layer_feature_model.predict(imgs)

In [33]:
len(each_layer_feature_model.outputs), len(features)

(296, 296)

In [34]:
features[0].shape

(10, 127, 127, 32)

In [35]:
i = 103
output = each_layer_feature_model.outputs[i]
layer_features = features[i]
get_shape(output), layer_features.shape

(array([ 14,  14, 128]), (10, 14, 14, 128))

In [8]:
feature_model

In [13]:
generator = Generator(os.path.join(absp, 'test.txt'), colored=images_are_colored, batch_size=batch_size)

In [14]:
features = feature_model.predict_generator(generator, workers=4, use_multiprocessing=True, verbose=1)

KeyboardInterrupt: 

In [ ]:
features.shape